# Простые счётчики на службе векторной семантики
## Скачиваем текст

In [ ]:
import urllib
from urllib import request

text_data = request.urlopen("http://www.gutenberg.org/files/1399/1399-0.txt").read().decode("UTF-8") + " "
text_data += request.urlopen("https://archive.org/stream/warandpeace030164mbp/warandpeace030164mbp_djvu.txt").read().decode("UTF-8")

text_data[:100]

## Убираем теги

In [ ]:
import re

clean_data = re.sub("<[^>]*>", " ", text_data)
clean_data = re.sub("\s+", " ", text_data)
clean_data[:100]

## Бьём текст на предложения

In [ ]:
import nltk
import nltk.data

# загружаем токенизатор
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
sentences = sent_detector.tokenize(clean_data.strip())

print("Total number of sentences:", len(sentences))

sentences[:2]

## Разбиваем предложения на токены

In [ ]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
import re

nltk.download('wordnet')

# можно попробовать и какой-нибудь другой
lemmatizer = WordNetLemmatizer()

# можно попробовать и какой-нибудь другой
tokenizer = TweetTokenizer()
splitted = []

for sent in sentences:
    splitted.append([lemmatizer.lemmatize(w).lower() for w in tokenizer.tokenize(sent) if re.match("^[A-Za-z'-]+$", w)])


## Своеобразный способ построить словарь

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '), min_df=25)
term_doc_matrix = vectorizer.fit_transform([" ".join(sentence) for sentence in splitted])

term2id = vectorizer.vocabulary_
id2term = {v: k for k, v in term2id.items()}

print("Term-document matrix shape:", term_doc_matrix.shape)
print("Vocabulary samples:", list(term2id.items())[:5])

## Посчитаем частоты термов, используя построенную матрицу

In [ ]:
term_counts = term_doc_matrix.sum(axis=0).A1
print(term_counts.shape)

---- 

### Опциональное задание: выбросьте слишком частотные и слишком редкие термы

----

In [ ]:
#todo: hint: надо использовать term_counts, term2id + перестроить все матрицы

## Строим term-context matrix

In [ ]:
import numpy as np
import scipy.sparse as sp

half_window = 5

X = np.zeros(shape=(len(term2id), len(term2id)))


for sentence in splitted:
    
    # бежим с индексом по предложению
    for i in range(len(sentence)):        
        current_word = sentence[i]      
        
        # если слова нет в словаре, ничего для него не считаем
        if current_word in term2id:
            word_idx = term2id[current_word]

            for c in range(-half_window, half_window):
                current_idx = i + c

                # проверяем, не наткнулись ли на границы предложения
                if 0 <= current_idx < len(sentence):
                    context_word = sentence[current_idx]
                    
                    if context_word in term2id:                    
                        context_idx = term2id[context_word]
                        X[word_idx, context_idx] += 1

print("Sparsity of the term-context matrix", len(X.nonzero()[0]) / (X.shape[0] ** 2))

## Функция поиска ближайших K

In [ ]:
from scipy.spatial.distance import cdist

def dict_k_closest(M, term_dict, inverse_term_dict, k=5):
    """
        :param M -- матрица векторых представлений
        :param term_dict -- слово2индекс
        :param inverse_term_dict -- индекс2слово
        :param k -- число ближайших соседей для выдачи
    """
    
    print("Computing all distances... (takes some time)")    
    distances = cdist(M, M, "cosine")
    sorted_by_dist_k = np.argsort(distances, axis=1)[:, :k]
    
    results = {}
    
    for term in term_dict:
        row_id = term_dict[term]
        similar = [inverse_term_dict[i] for i in sorted_by_dist_k[row_id, :]]
        results[term] = similar   
        
    return results

## Смотрим глазами на наши успехи

In [ ]:
similar_terms = dict_k_closest(X, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

## Ну как вам?

(Правильный ответ: не очень)


### PMI: Pointwise Mutual Information
    

In [ ]:
X_PMI = X.copy()
total_bicount = X_PMI.sum()
total_unicount = term_counts.sum()

# p(x,y) / p(x) / p(y)
X_PMI = (X_PMI / total_bicount) / (term_counts[:, None] + 0.000001) / (term_counts[None, :] + 0.000001)  * total_unicount ** 2
X_PMI = np.where(X_PMI > 0, np.log2(X_PMI), 0)

In [ ]:
similar_terms = dict_k_closest(X_PMI, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

## Positive PMI

In [ ]:
X_PPMI = X_PMI.copy()
X_PPMI[X_PPMI < 0] = 0.0

In [ ]:
similar_terms = dict_k_closest(X_PPMI, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

### Ну как?
Должно стать лучше

## Применим Truncated SVD

In [ ]:
# U, S, Vh = np.linalg.svd(X_PPMI, full_matrices=False)
# U.shape, S.shape, Vh.shape

In [ ]:
from  sklearn.decomposition import TruncatedSVD

factorizer = TruncatedSVD(n_components=200, random_state=0)

# матрица слов
W = factorizer.fit_transform(X_PPMI)
print(W.shape)

# матрица контекстов
C = factorizer.components_
print(C.shape)

In [ ]:
similar_terms = dict_k_closest(W, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

### Попробуем разложить в произведение двух матриц

In [ ]:
from  sklearn.decomposition import NMF

factorizer = NMF(n_components=200, random_state=0)

# матрица слов
W = factorizer.fit_transform(X_PPMI)
print(W.shape)

# матрица контекстов
C = factorizer.components_
print(C.shape)

In [ ]:
similar_terms = dict_k_closest(W, term2id, id2term)

In [ ]:
for key in list(similar_terms)[:10]:
    print(key, ":", " ".join(similar_terms[key][1:]))

##  Задача техническая: сохранить векторы в CSV
---- 

# Посмотрим, как можно оценивать качество
Отличный источник, горячо рекомендуется
https://github.com/EloiZ/embedding_evaluation

Надо склонировать репозиторий, загрузить датасеты с помощью
`download_benchmarks.py`

In [ ]:
import os

os.environ["EMBEDDING_EVALUATION_DATA_PATH"] = "embedding_evaluation/data/"

import embedding_evaluation
from embedding_evaluation.evaluate import Evaluation
from embedding_evaluation.load_embedding import load_embedding_textfile

def eval_word_vectors(path):
    # Load embeddings as a dictionnary {word: embed} where embed is a 1-d numpy array.
    embeddings = load_embedding_textfile(textfile_path=path)

    # Load and process evaluation benchmarks
    evaluation = Evaluation() 

    return evaluation.evaluate(embeddings)

## Задача: оценить качество сохранённых моделей, попытаться его улучшить

In [ ]:
mymodelresults = eval_word_vectors("trali-vali.csv")

mymodelresults